In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

# Load labels
df = pd.read_csv('/kaggle/input/iitgai-ovha-23/train_labels.csv')  

# Assuming images are in the '/kaggle/input/iitgai-ovha-23/images/' folder
image_paths = df['image_name'].apply(lambda x: f'/kaggle/input/iitgai-ovha-23/images/{x}')

# Function to load and preprocess images
def load_and_preprocess_image(image_path):
    img = keras.preprocessing.image.load_img(image_path, target_size=(224, 224))
    img = keras.preprocessing.image.img_to_array(img)
    img = keras.applications.resnet50.preprocess_input(img)
    return img

images = np.array([load_and_preprocess_image(path) for path in image_paths])

# Function to parse points from the 'points' column
def parse_points(points_str):
    try:
        # If points are already in list format as strings, eval() will convert them to a list
        if isinstance(points_str, str):
            return eval(points_str)
        else:
            return points_str
    except Exception as e:
        print(f"Error parsing points: {e}, points_str: {points_str}")
        return None

# Apply the parse_points function and remove problematic points
labels = df['points'].apply(parse_points)
labels = labels.dropna()

# Convert labels to numpy array
labels = np.array(labels.tolist())

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)

# Load pre-trained ResNet model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom layers
model = keras.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(8)  # 8 output values for the coordinates
])

# Train the model for 200 epochs
model.compile(optimizer=Adam(learning_rate=0.0001), loss='mean_squared_error')
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=200, batch_size=32)

# Save the model
model.save('/kaggle/working/icu_machine_detection_model.h5') 

# Print the final cost function (loss) of train and validation set
train_loss = history.history['loss'][-1]
val_loss = history.history['val_loss'][-1]
print(f'Final Training Loss: {train_loss:.4f}')
print(f'Final Validation Loss: {val_loss:.4f}')


In [ ]:
import numpy as np
import pandas as pd
from tensorflow import keras

# Load the pre-trained model
model = keras.models.load_model('/kaggle/working/icu_machine_detection_model.h5')  # Adjust the path if needed

# Load the sample submission file
sample_submission = pd.read_csv('/kaggle/input/iitgai-ovha-23/sample_submission.csv')

# Assuming test images are in the '/kaggle/input/iitgai-ovha-23/test_images/' folder
test_image_paths = sample_submission['image_name'].apply(lambda x: f'/kaggle/input/iitgai-ovha-23/test_images/{x}')

# Function to load and preprocess test images
def load_and_preprocess_test_image(image_path):
    img = keras.preprocessing.image.load_img(image_path, target_size=(224, 224))
    img = keras.preprocessing.image.img_to_array(img)
    img = keras.applications.resnet50.preprocess_input(img)
    return img

test_images = np.array([load_and_preprocess_test_image(path) for path in test_image_paths])

# Predict coordinates for test images
predicted_coordinates = model.predict(test_images)

# Create a DataFrame with image names and predicted coordinates
submission_df = pd.DataFrame({
    'image_name': sample_submission['image_name'],
    'points': [f'[{coord[0]},{coord[1]},{coord[2]},{coord[3]}' for coord in predicted_coordinates]
})

# Save the submission file
submission_df.to_csv('/kaggle/working/submission.csv', index=False) 

# Optionally, you can also print the first few rows of the submission DataFrame
print(submission_df.head())


In [1]:
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load labels
df = pd.read_csv('/kaggle/input/iitgai-ovha-23/train_labels.csv')  

# Assuming images are in the '/kaggle/input/iitgai-ovha-23/images/' folder
image_paths = df['image_name'].apply(lambda x: f'/kaggle/input/iitgai-ovha-23/images/{x}')

# Function to load and preprocess images
def load_and_preprocess_image(image_path):
    img = keras.preprocessing.image.load_img(image_path, target_size=(224, 224))
    img = keras.preprocessing.image.img_to_array(img)
    img = keras.applications.resnet50.preprocess_input(img)
    return img

images = np.array([load_and_preprocess_image(path) for path in image_paths])

# Function to parse points from the 'points' column
def parse_points(points_str):
    try:
        # If points are already in list format as strings, eval() will convert them to a list
        if isinstance(points_str, str):
            return eval(points_str)
        else:
            return points_str
    except Exception as e:
        print(f"Error parsing points: {e}, points_str: {points_str}")
        return None

# Apply the parse_points function and remove problematic points
labels = df['points'].apply(parse_points)
labels = labels.dropna()

# Convert labels to numpy array
labels = np.array(labels.tolist())

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)

# Apply data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Initialize an empty list to store augmented images and labels
augmented_images = []
augmented_labels = []

# Apply data augmentation to each image and its corresponding label
for image, label in zip(X_train, y_train):
    image = np.expand_dims(image, axis=0)  # Add batch dimension
    augmented_images.extend(datagen.flow(image).next())
    augmented_labels.extend([label] * len(datagen.flow(image).next()))

# Convert the lists of augmented images and labels back to numpy arrays
augmented_images = np.array(augmented_images)
augmented_labels = np.array(augmented_labels)

# Combine original and augmented data
X_combined = np.vstack((X_train, augmented_images))
y_combined = np.vstack((y_train, augmented_labels))  # Assuming labels are 2D

# Load pre-trained ResNet model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom layers
model = keras.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(8)  # 8 output values for the coordinates
])

# Train the model for 200 epochs
model.compile(optimizer=Adam(learning_rate=0.0001), loss='mean_squared_error')
history = model.fit(X_combined, y_combined, validation_data=(X_val, y_val), epochs=200, batch_size=32)

# Save the model
model.save('/kaggle/working/icu_machine_detection_model.h5') 

# Print the final cost function (loss) of train and validation set
train_loss = history.history['loss'][-1]
val_loss = history.history['val_loss'][-1]
print(f'Final Training Loss: {train_loss:.4f}')
print(f'Final Validation Loss: {val_loss:.4f}')


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Epoch 1/200
90/90 [==============================] - 57s 202ms/step - loss: 288880.6562 - val_loss: 89484.2422
Epoch 2/200
90/90 [==============================] - 16s 180ms/step - loss: 261366.2656 - val_loss: 195906.2500
Epoch 3/200
90/90 [==============================] - 17s 187ms/step - loss: 240578.2188 - val_loss: 209384.0938
Epoch 4/200
90/90 [==============================] - 16s 180ms/step - loss: 221286.2031 - val_loss: 207784.9844
Epoch 5/200
90/90 [==============================] - 16s 180ms/step - loss: 203694.4062 - val_loss: 197483.5625
Epoch 6/200
90/90 [==============================] - 16s 179ms/step - loss: 186282.0312 - val_loss: 160822.8438
Epoch 7/200
90/90 [==============================] - 16s 180ms/step - loss: 170749.6406 - val_loss: 150858.6719
Epoch 8/200
90/90 [==============================] - 16s 179ms/step - loss: 154913.6250 - val_loss: 147905.2812
Epoch 9/200
90/90 [==============================] - 16s 180ms/step - loss: 140346.2656 - val_loss: 15126

In [2]:
import numpy as np
import pandas as pd
from tensorflow import keras

# Load the pre-trained model
model = keras.models.load_model('/kaggle/working/icu_machine_detection_model.h5')  # Adjust the path if needed

# Load the sample submission file
sample_submission = pd.read_csv('/kaggle/input/iitgai-ovha-23/sample_submission.csv')

# Assuming test images are in the '/kaggle/input/iitgai-ovha-23/test_images/' folder
test_image_paths = sample_submission['image_name'].apply(lambda x: f'/kaggle/input/iitgai-ovha-23/test_images/{x}')

# Function to load and preprocess test images
def load_and_preprocess_test_image(image_path):
    img = keras.preprocessing.image.load_img(image_path, target_size=(224, 224))
    img = keras.preprocessing.image.img_to_array(img)
    img = keras.applications.resnet50.preprocess_input(img)
    return img

test_images = np.array([load_and_preprocess_test_image(path) for path in test_image_paths])

# Predict coordinates for test images
predicted_coordinates = model.predict(test_images)

# Create a DataFrame with image names and predicted coordinates
submission_df = pd.DataFrame({
    'image_name': sample_submission['image_name'],
    'points': [f"[{coord[0]:.1f}, {coord[1]:.1f}, {coord[2]:.1f}, {coord[3]:.1f}, {coord[4]:.1f}, {coord[5]:.1f}, {coord[6]:.1f}, {coord[7]:.1f}]" for coord in predicted_coordinates]
})

# Save the submission DataFrame as a CSV file
submission_df.to_csv('/kaggle/working/icu_machine_detection_submission.csv', index=False)


8/8 [==============================] - 2s 123ms/step


In [1]:
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler
from tensorflow.keras.regularizers import l2
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load labels
df = pd.read_csv('/kaggle/input/iitgai-ovha-23/train_labels.csv')  

# Assuming images are in the '/kaggle/input/iitgai-ovha-23/images/' folder
image_paths = df['image_name'].apply(lambda x: f'/kaggle/input/iitgai-ovha-23/images/{x}')

# Function to load and preprocess images
def load_and_preprocess_image(image_path):
    img = keras.preprocessing.image.load_img(image_path, target_size=(224, 224))
    img = keras.preprocessing.image.img_to_array(img)
    img = keras.applications.resnet50.preprocess_input(img)
    return img

images = np.array([load_and_preprocess_image(path) for path in image_paths])

# Function to parse points from the 'points' column
def parse_points(points_str):
    try:
        # If points are already in list format as strings, eval() will convert them to a list
        if isinstance(points_str, str):
            return eval(points_str)
        else:
            return points_str
    except Exception as e:
        print(f"Error parsing points: {e}, points_str: {points_str}")
        return None

# Apply the parse_points function and remove problematic points
labels = df['points'].apply(parse_points)
labels = labels.dropna()

# Convert labels to numpy array
labels = np.array(labels.tolist())

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)

# Apply data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Initialize an empty list to store augmented images and labels
augmented_images = []
augmented_labels = []

# Apply data augmentation to each image and its corresponding label
for image, label in zip(X_train, y_train):
    image = np.expand_dims(image, axis=0)  # Add batch dimension
    augmented_images.extend(datagen.flow(image).next())
    augmented_labels.extend([label] * len(datagen.flow(image).next()))

# Convert the lists of augmented images and labels back to numpy arrays
augmented_images = np.array(augmented_images)
augmented_labels = np.array(augmented_labels)

# Combine original and augmented data
X_combined = np.vstack((X_train, augmented_images))
y_combined = np.vstack((y_train, augmented_labels))  # Assuming labels are 2D

# Define a function to adjust learning rate after 180 epochs
def lr_scheduler(epoch):
    if epoch < 180:
        return 0.0001  # Keep learning rate as 0.0001 for the first 180 epochs
    else:
        return 0.00001  # Change learning rate to 0.00001 after 180 epochs

# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
lr_schedule = LearningRateScheduler(lr_scheduler)

# Load pre-trained ResNet model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add L2 regularization to the dense layer
model = keras.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(8, kernel_regularizer=l2(0.00001))  # Add L2 regularization with alpha=0.00001
])

# Train the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='mean_squared_error')
history = model.fit(X_combined, y_combined, validation_data=(X_val, y_val), epochs=200, batch_size=32, callbacks=[early_stopping, lr_schedule])

# Save the model
model.save('/kaggle/working/icu_machine_detection_model.h5') 

# Print the final cost function (loss) of train and validation set
train_loss = history.history['loss'][-1]
val_loss = history.history['val_loss'][-1]
print(f'Final Training Loss: {train_loss:.4f}')
print(f'Final Validation Loss: {val_loss:.4f}')


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


94765736/94765736 [==============================] - 3s 0us/step
Epoch 1/200
90/90 [==============================] - 63s 206ms/step - loss: 289128.6562 - val_loss: 123782.2422 - lr: 1.0000e-04
Epoch 2/200
90/90 [==============================] - 17s 185ms/step - loss: 262298.7812 - val_loss: 183934.0156 - lr: 1.0000e-04
Epoch 3/200
90/90 [==============================] - 17s 186ms/step - loss: 241923.8594 - val_loss: 230168.5156 - lr: 1.0000e-04
Epoch 4/200
90/90 [==============================] - 16s 178ms/step - loss: 222635.3125 - val_loss: 194361.2656 - lr: 1.0000e-04
Epoch 5/200
90/90 [==============================] - 16s 179ms/step - loss: 204733.9375 - val_loss: 195221.1094 - lr: 1.0000e-04
Epoch 6/200
90/90 [==============================] - 16s 178ms/step - loss: 187974.2031 - val_loss: 163754.0156 - lr: 1.0000e-04
Epoch 7/200
90/90 [==============================] - 16s 178ms/step - loss: 171555.1562 - val_loss: 141836.8281 - lr: 1.0000e-04
Epoch 8/200
90/90 [=============

In [2]:
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler
from tensorflow.keras.regularizers import l2
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load labels
df = pd.read_csv('/kaggle/input/iitgai-ovha-23/train_labels.csv')  

# Assuming images are in the '/kaggle/input/iitgai-ovha-23/images/' folder
image_paths = df['image_name'].apply(lambda x: f'/kaggle/input/iitgai-ovha-23/images/{x}')

# Function to load and preprocess images
def load_and_preprocess_image(image_path):
    img = keras.preprocessing.image.load_img(image_path, target_size=(224, 224))
    img = keras.preprocessing.image.img_to_array(img)
    img = keras.applications.resnet50.preprocess_input(img)
    return img

images = np.array([load_and_preprocess_image(path) for path in image_paths])

# Function to parse points from the 'points' column
def parse_points(points_str):
    try:
        # If points are already in list format as strings, eval() will convert them to a list
        if isinstance(points_str, str):
            return eval(points_str)
        else:
            return points_str
    except Exception as e:
        print(f"Error parsing points: {e}, points_str: {points_str}")
        return None

# Apply the parse_points function and remove problematic points
labels = df['points'].apply(parse_points)
labels = labels.dropna()

# Convert labels to numpy array
labels = np.array(labels.tolist())

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)

# Load pre-trained ResNet model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add L2 regularization to the dense layer
model = keras.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(8, kernel_regularizer=l2(0.00001))  # Add L2 regularization with alpha=0.00001
])

# Load the previously saved model
model.load_weights('/kaggle/working/icu_machine_detection_model.h5')

# Define a function to adjust learning rate after 180 epochs
def lr_scheduler(epoch):
    if epoch < 180:
        return 0.0001  # Keep learning rate as 0.0001 for the first 180 epochs
    else:
        return 0.00001  # Change learning rate to 0.00001 after 180 epochs

# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
lr_schedule = LearningRateScheduler(lr_scheduler)

# Train the model starting from epoch 70
model.compile(optimizer=Adam(learning_rate=0.0001), loss='mean_squared_error')
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=200, initial_epoch=70, batch_size=32, callbacks=[early_stopping, lr_schedule])

# Save the model
model.save('/kaggle/working/icu_machine_detection_model.h5') 

# Print the final cost function (loss) of train and validation set
train_loss = history.history['loss'][-1]
val_loss = history.history['val_loss'][-1]
print(f'Final Training Loss: {train_loss:.4f}')
print(f'Final Validation Loss: {val_loss:.4f}')


Epoch 71/200
45/45 [==============================] - 45s 223ms/step - loss: 539.9044 - val_loss: 1373.1417 - lr: 1.0000e-04
Epoch 72/200
45/45 [==============================] - 8s 189ms/step - loss: 404.1269 - val_loss: 860.9225 - lr: 1.0000e-04
Epoch 73/200
45/45 [==============================] - 8s 187ms/step - loss: 348.7886 - val_loss: 1219.3771 - lr: 1.0000e-04
Epoch 74/200
45/45 [==============================] - 8s 187ms/step - loss: 406.4791 - val_loss: 2000.8280 - lr: 1.0000e-04
Epoch 75/200
45/45 [==============================] - 8s 186ms/step - loss: 363.2711 - val_loss: 1429.9757 - lr: 1.0000e-04
Epoch 76/200
45/45 [==============================] - 8s 186ms/step - loss: 335.4020 - val_loss: 2745.1133 - lr: 1.0000e-04
Epoch 77/200
45/45 [==============================] - 9s 200ms/step - loss: 284.2615 - val_loss: 1813.9094 - lr: 1.0000e-04
Epoch 78/200
45/45 [==============================] - 8s 186ms/step - loss: 302.0045 - val_loss: 911.8030 - lr: 1.0000e-04
Epoch 79/

In [2]:
# import numpy as np
# import pandas as pd
# from tensorflow import keras
# from tensorflow.keras import layers
# from tensorflow.keras.applications import ResNet50
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler
# from tensorflow.keras.regularizers import l2
# from sklearn.model_selection import train_test_split
# from tensorflow.keras.preprocessing.image import ImageDataGenerator

# # Load labels
# df = pd.read_csv('/kaggle/input/iitgai-ovha-23/train_labels.csv')  

# # Assuming images are in the '/kaggle/input/iitgai-ovha-23/images/' folder
# image_paths = df['image_name'].apply(lambda x: f'/kaggle/input/iitgai-ovha-23/images/{x}')

# # Function to load and preprocess images
# def load_and_preprocess_image(image_path):
#     img = keras.preprocessing.image.load_img(image_path, target_size=(224, 224))
#     img = keras.preprocessing.image.img_to_array(img)
#     img = keras.applications.resnet50.preprocess_input(img)
#     return img

# images = np.array([load_and_preprocess_image(path) for path in image_paths])

# # Function to parse points from the 'points' column
# def parse_points(points_str):
#     try:
#         # If points are already in list format as strings, eval() will convert them to a list
#         if isinstance(points_str, str):
#             return eval(points_str)
#         else:
#             return points_str
#     except Exception as e:
#         print(f"Error parsing points: {e}, points_str: {points_str}")
#         return None

# # Apply the parse_points function and remove problematic points
# labels = df['points'].apply(parse_points)
# labels = labels.dropna()

# # Convert labels to numpy array
# labels = np.array(labels.tolist())

# # Split data into training and validation sets
# X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)

# # Load pre-trained ResNet model
# base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# # Add L2 regularization to the dense layer
# model = keras.Sequential([
#     base_model,
#     layers.GlobalAveragePooling2D(),
#     layers.Dense(8, kernel_regularizer=l2(0.00001))  # Add L2 regularization with alpha=0.00001
# ])

# # Load the previously saved model
# model.load_weights('/kaggle/working/icu_machine_detection_model.h5')

# # Define a function to adjust learning rate after 180 epochs
# def lr_scheduler(epoch):
#     if epoch < 180:
#         return 0.0001  # Keep learning rate as 0.0001 for the first 180 epochs
#     else:
#         return 0.00001  # Change learning rate to 0.00001 after 180 epochs

# # Define callbacks
# early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
# lr_schedule = LearningRateScheduler(lr_scheduler)

# # Train the model starting from epoch 92
# model.compile(optimizer=Adam(learning_rate=0.0001), loss='mean_squared_error')
# history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=200, initial_epoch=92, batch_size=32, callbacks=[early_stopping, lr_schedule])

# # Save the model
# model.save('/kaggle/working/icu_machine_detection_model.h5') 

# # Print the final cost function (loss) of train and validation set
# train_loss = history.history['loss'][-1]
# val_loss = history.history['val_loss'][-1]
# print(f'Final Training Loss: {train_loss:.4f}')
# print(f'Final Validation Loss: {val_loss:.4f}')
